In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn import datasets


In [2]:
iris = datasets.load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [3]:
df=pl.from_pandas(df)
df

sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
f64,f64,f64,f64,f64
5.1,3.5,1.4,0.2,0.0
4.9,3.0,1.4,0.2,0.0
4.7,3.2,1.3,0.2,0.0
4.6,3.1,1.5,0.2,0.0
5.0,3.6,1.4,0.2,0.0
…,…,…,…,…
6.7,3.0,5.2,2.3,2.0
6.3,2.5,5.0,1.9,2.0
6.5,3.0,5.2,2.0,2.0


In [4]:
df.columns

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)',
 'target']

In [5]:
from sklearn.model_selection import train_test_split

X=df.select('sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)')
y=df.select('target')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)


In [6]:
from sklearn.linear_model import LogisticRegression

model=LogisticRegression()

model.fit(X=X_train,y=y_train)

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

fai confusion matrix e altre metriche per classificazione es: sensibilità

In [7]:
from sklearn import metrics

y_pred=model.predict(X_test)

metrics.confusion_matrix(y_true=y_test,y_pred=y_pred)

array([[13,  0,  0],
       [ 0, 14,  1],
       [ 0,  2,  8]])

In [8]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
# see how model confusion matrix varies with threshold
bestmodel = model

#labels = y_test.sort_values().drop_duplicates().to_list()
labels = ['0','1','2']

def make_confusion_matrix(model, threshold=0.5):
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (bestmodel.predict(X_test) is done automatically with a threshold of 0.5)
    y_pred = (bestmodel.predict_proba(X_test)[:, 1] >= threshold)
    conf = confusion_matrix(y_test, y_pred)
    plt.figure(figsize = [6,6])
    sns.heatmap(conf, cmap='Blues', annot=True, square=True, cbar=False,
           xticklabels=labels,
           yticklabels=labels,fmt='.3g');
    plt.xlabel('Actual')
    plt.ylabel('Prediction')
    plt.show() # This is part that needs to be added to fix the issue

from ipywidgets import interactive, FloatSlider
interactive(lambda threshold: make_confusion_matrix(bestmodel, threshold), threshold=(0.0,1.0,0.01))

interactive(children=(FloatSlider(value=0.5, description='threshold', max=1.0, step=0.01), Output()), _dom_cla…

In [9]:
from sklearn.metrics import classification_report
target_names = ['0', '1','2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.88      0.93      0.90        15
           2       0.89      0.80      0.84        10

    accuracy                           0.92        38
   macro avg       0.92      0.91      0.92        38
weighted avg       0.92      0.92      0.92        38



In [10]:
from matplotlib import axes
y_pred_proba = model.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.plot([0, 1], [0, 1])
plt.legend(loc=4)
plt.show()

ValueError: multiclass format is not supported